In [53]:
DEBUG = True
model_name = 'mobilenetv3_large_100'

In [54]:
from PIL import Image
import torchvision.transforms as transforms
import torch

In [55]:
PATH = 'model/' + model_name + '.pth'
model = torch.load(PATH)

In [63]:
def recognize(image_path):
    """Print top 5 possiblilities of the given image.

    Args:
      image_path: the path of the image

    Returns:
    """
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224,224)),
    ])
    tensor = transform(img).unsqueeze(0)
    
    if DEBUG:
        print(img.size)
        print(tensor.shape)
    
    model.eval()
    with torch.no_grad():
        out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    
    if DEBUG:
        print(probabilities)
    
    with open("imagenet_classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())

In [62]:
recognize('image/dog.jpg')

(1546, 1213)
torch.Size([1, 3, 224, 224])
tensor([[-1.4882e-01,  1.0950e+00, -9.8916e-01, -1.3882e+00, -7.5383e-01,
          1.1254e-01,  9.4546e-01,  1.0027e+00,  2.5040e+00, -1.0097e+00,
         -4.1081e-01, -3.2439e-01, -5.8317e-02,  2.1157e-01,  5.4429e-02,
          1.0515e+00,  3.5043e-01,  3.3909e-02,  1.5764e-01, -2.9059e-01,
          3.9213e-01,  1.1641e-01, -1.1553e-01,  6.6436e-01,  3.4093e-01,
          1.0334e-01, -3.3556e-01,  2.1519e-01,  2.0324e-01,  1.0238e+00,
          5.4629e-01,  1.1877e-02,  1.9895e-01, -9.9633e-01, -4.1305e-01,
          1.8186e-01,  1.5377e-01,  1.1269e+00, -8.7311e-01, -9.2705e-02,
          2.7787e-01, -3.2287e-02, -2.8847e-01,  6.9856e-01, -2.5396e-01,
          1.4448e-01, -2.4098e-01,  8.9171e-02,  1.6734e-01,  1.0499e-01,
         -7.9748e-01, -5.2126e-01,  5.2495e-01, -1.3978e-01,  2.4202e-01,
          5.0929e-01, -3.7757e-01,  7.1074e-01,  1.6395e-01,  7.9339e-01,
          1.3117e+00, -4.2532e-01,  5.9482e-01, -4.4038e-01,  7.7393e-